In [1]:
import itertools
import math
from itertools import islice
from pprint import pprint

import numpy as np
import opentrons.simulate

protocol = opentrons.simulate.get_protocol_api("2.12")
protocol.home()

%run opentrons_titrate_functions.ipynb

/Users/timkench/.opentrons/robot_settings.json not found. Loading defaults
/Users/timkench/.opentrons/deck_calibration.json not found. Loading defaults


In [2]:
tiprack_20a = protocol.load_labware("opentrons_96_tiprack_20ul", location="7")
tiprack_20b = protocol.load_labware("opentrons_96_tiprack_20ul", location="8")
tiprack_20c = protocol.load_labware("opentrons_96_tiprack_20ul", location="9")

tiprack_300a = protocol.load_labware("opentrons_96_tiprack_300ul", location="10")
tiprack_300b = protocol.load_labware("opentrons_96_tiprack_300ul", location="11")

m20 = protocol.load_instrument(
    "p20_multi_gen2", "left", tip_racks=[tiprack_20a, tiprack_20b, tiprack_20c]
)
m300 = protocol.load_instrument(
    "p300_multi_gen2", "right", tip_racks=[tiprack_300a, tiprack_300b]
)

labware_96w_200ul = "biorad_96_wellplate_200ul_pcr"
labware_12w_22ml = "nest_96_wellplate_2ml_deep"
labware_384w_20ul = "appliedbiosystemsmicroamp_384_wellplate_40ul"

res_plate_ot = protocol.load_labware(labware_12w_22ml, "1")
stock_plate_ot = protocol.load_labware(labware_96w_200ul, "2")
dil_plate_ot = protocol.load_labware(labware_96w_200ul, "3")
measure_plate_ot = protocol.load_labware(labware_384w_20ul, "4")

In [3]:
# set up plates

res_plate = ResPlate(reagents={"buffer": 0, "reagent": 1})
res_plate.assign_ot(res_plate_ot)

stock_plate = GeneralPlate(active_cols=(0, 4), start_conc=5000, start_vol=2)
stock_plate.assign_ot(stock_plate_ot)

dil_plate = GeneralPlate(map_to=stock_plate, end_conc=40, end_vol=100)
dil_plate.assign_ot(dil_plate_ot)
dil_plate.make_transfer_map()

measure_plate = MeasurePlate(map_to=dil_plate, points=8, repeats=3)
measure_plate.assign_ot(measure_plate_ot)
measure_plate.make_transfer_map()

# set up pipettes and protocol

m20_c = CustomPipette(m20, name="m20_c")
m300_c = CustomPipette(m300, name="m300_c")

CP = CustomProtocol(
    solvent_location=res_plate.reagents["buffer"], m20=m20_c, m300=m300_c
)

# do titration part 1

CP.add(plate=stock_plate, vol=100)
CP.dilution(start_plate=stock_plate, end_plate=dil_plate)
CP.serial_dilute(plate=measure_plate, vol=9, sd_factor=0.5)
CP.add(
    plate=measure_plate,
    source=res_plate.reagents["reagent"],
    vol=9,
    top_dispense="Yes",
)

# titration part 2 after new plate added to opentrons

measure_plate.make_transfer_map()
CP.serial_dilute(plate=measure_plate, vol=9, sd_factor=0.5)
CP.add(
    plate=measure_plate,
    source=res_plate.reagents["reagent"],
    vol=9,
    top_dispense="Yes",
)


This will add the specified source to the active columns of the plate using the m300_c pipette.
The plate will be diluted to 100.0 µM


Do you want to continue? (Y/N):  y


continuing...
done
plate diluted to 100.0 µM
This will transfer 40.0 µL and combine it with 60.0 µL of solvent for a end concentration of 40 µM.


Do you want to continue? (Y/N):  y


continuing...
done


TypeError: 'float' object is not callable

In [5]:
dil_plate.transfer_map

[[A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2,
  A1 of Bio-Rad 96 Well Plate 200 µL PCR on 3],
 [A2 of Bio-Rad 96 Well Plate 200 µL PCR on 2,
  A2 of Bio-Rad 96 Well Plate 200 µL PCR on 3],
 [A3 of Bio-Rad 96 Well Plate 200 µL PCR on 2,
  A3 of Bio-Rad 96 Well Plate 200 µL PCR on 3],
 [A4 of Bio-Rad 96 Well Plate 200 µL PCR on 2,
  A4 of Bio-Rad 96 Well Plate 200 µL PCR on 3]]